# SQL

SQLite3を起動するには、Macの方はターミナルを起動し、次のようにします。
sqlite3

sqliteでデータベースを作成するには、sqlite起動時に次のように実行します。

sqlite3 sample_db;

テーブルを作成するにはcreateを利用します。
例えば、sample_tableという名前のテーブルを作成するには次のようにします。

create table sample_table(id, name);

テーブルが作成されたかを確認するには次のコマンドを実行します。

.tables

また、.schemaを使うことでテーブルの構造を取得可能です。

データの追加
INSERT文を使うことで、データを追加することが出来ます。

insert into sample_table values(1, 'apple');
insert into sample_table values(2, 'grapes');
insert into sample_table values(3, 'orange');

データの取得
データを取得するにはselectを用います。

select * from sample_table;

1|apple
2|grapes
3|orange

# CSV読み込み方法

CSV読み込み方法
SQLite3でCSVファイルを読み込むには次のようにします。

sqlite> .mode csv
sqlite> .import csvファイルのパス　作成したいパス　作成したいテーブル名

読み込み、作成までの方法には、例えば、次のようにkpi.csvというcsvファイルを/Users/cyberbrain/Desktop/kpi.csvがある場合に、次のようにします。

.import /Users/cyberbrain/Desktop/kpi.csv kpi

テーブルが正しく作成されたかどうかは次のようにします。

.tables

上記を実行し、kpiというテーブルが出力されていれば正しく動作が出来ています。

CSVファイルの書き出し

次に、先ほど読み込んだCSVファイルを別のファイル名で書き出して見ます。
ここで注意なのは、最後の.quitを実行することで初めて書き込みが行われます。
それでは順番に見ていきましょう。

sqlite> .headers on
sqlite> .mode csv
sqlite> .output data.csv
sqlite> SELECT * from kpi;
sqlite> .quit

それでは、data.csvというファイル名を出力してみましょう。
手順としては５通りです。

まず、.headers onを入力します。
これは結果セットのヘッダーをオンにしています。
これによりヘッダー部分も書き込まれます。（ヘッダーの部分の書き込みが必要ない場合は、この記述は必要ありません。）

次に.mode csvを記述します。

次に、.output ファイル名とし、書き出したいcsvファイル名を記載します。
最後に、書き出したい中身を抽出します。
ここで、全データを書き出したい場合には、SELECT * from テーブル名;とします。

最後に、必ず、.quitを実行しsqlite3を終了し、書き出されたcsvファイルを確認してみてください。
うまく書き出されていたら成功です。

# PythonからSQLiteを利用する

In [2]:
import sqlite3

データベースを作成する
それではデータベースを作成していきます。
次のプログラムを打ち込みEnterキーで実行します。

In [3]:
db = sqlite3.connect("sample.db")

テーブルを作成する
それでは、データベースを作成することが出来ましたので、次にテーブルを作成します。
次のコードを打ち込み実行してください。

In [4]:
cursor = db.cursor()
cursor.execute("CREATE TABLE users(user_id, user_name)");

作成したテーブルに新規データを追加する
先ほどusersテーブルを作成しました。
しかし、作成は出来ましたがまだ何もデータが存在しない状態です。
ここでは作成したusersテーブルに新規データを追加していきます。
それでは先ほどの続きに下記のプログラムを打ち込み実行してください。

In [6]:
cursor.execute("INSERT INTO users VALUES(1, 'python')")

上記を実行することで、新規にデータを追加することが可能です。
既にお気づきかもしれませんが、execute()のカッコ内にSQLを渡すことでそのSQLを実行することが出来ます。

さて、上記方法で新規データを追加することが出来ますが、本来であれば上記書き方は実務ではあまりされません。
というもの、SQLインジェクションという脆弱性を突かれる可能性があるためです。

SQLインジェクションとは、アプリケーションのセキュリティ上の不備を意図的に利用し、アプリケーションが想定しないSQL文を実行させることにより、データベースシステムを不正に操作する攻撃方法のこと。

SQLインジェクション対策として使われるのがプレースホルダというものがあります。
プレースホルダは、パラメータ部分を示す記号?のことですが、プレースホルダとして?を使うことで、値を安全に渡すことが可能です。

次のように記述可能です。

In [7]:
cursor.execute("INSERT INTO users(user_id, user_name) VALUES(?,?)",(2,'python3'))

上記方法で安全にデータを新規追加することが出来ました。
また、複数のデータを追加したい場合にはexecutemany()を使うことで、複数追加が出来ます。

In [8]:
data = [
    (3, "ruby"),
    (4, "r"),
    (5, "java")
]
cursor.executemany("INSERT INTO users VALUES(?,?)", data)

データを反映させる
ここまでの内容でデータベースの作成方法、テーブルの作成方法、SQLの実行方法を学びました。
ですが、実は先ほどのデータの追加はまだ反映されていません。
データを反映させるためには、commit()を実行します。
次のプログラムを打ち込み、実行してください。

In [9]:
db.commit()

大事なのは、execute()終えた後には必ずcommit()を実行する必要があります。
実行しないとデータが正しく反映されません。

データを取得する
それでは先ほどまでに追加したデータを取得してみましょう。
データを取得するには、execute()に加え、fetchall()と合わせて利用します。
次のプログラムを記述し実行してください。

In [10]:
cursor.execute('SELECT * FROM users')
cursor.fetchall()

[(1, 'python'), (2, 'python3'), (3, 'ruby'), (4, 'r'), (5, 'java')]

SELECT * FROM テーブル名という記述により、全てのデータを取得することが出来ます。
アスタリスク(*)は全てを意味します。
そしてfetchall()は全ての行を取得します。
取得したデータがPythonのリスト型で返ってきます。

他にも、fetchone()を用いてデータを1つ1つ取り出すことも可能です。
次のように記述します。